# Data Preprocessing (データ前処理)
本ノートブックでは、入力テキストのクレンジング、BERTモデル入力のためのトークン長フィルタリング、およびラベルベースのデータ分割を行います。

## 1. Setup & Model Initialization
必要なライブラリのインポートと、BERTモデル・トークナイザーの初期化を行います。

In [ ]:
import pandas as pd
import re
from tqdm import tqdm
import torch
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT tokenizer and model (bert-base-uncased)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', model_max_length=512)
model = BertModel.from_pretrained('bert-base-uncased')

# Set device to GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

## 2. Data Merging
テストデータにおける問題文（features）と正解ラベル（labels）を結合します。

In [ ]:
# Load raw datasets
df_label = pd.read_csv("test_labels.csv")
df_problem = pd.read_csv("test.csv")

# Merge labels and features on 'id' (inner join)
df_test = pd.merge(df_label, df_problem, how="inner", on="id")

# Export merged dataframe
df_test.to_csv("test_new.csv", index=False)

## 3. Text Normalization
不要な記号の除去や、短縮形（Contractions）の展開を行うための辞書と関数を定義します。

In [ ]:
# Dictionary for expanding English contractions
cList = {
  "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have",
  "'cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have",    "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
  "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would",
  "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
  "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
  "i'd": "i would", "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have",
  "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it had", "it'd've": "it would have",
  "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us",
  "ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not",
  "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
  "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
  "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
  "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
  "she'd": "she would", "she'd've": "she would have", "she'll": "she will",
  "she'll've": "she will have", "she's": "she is", "should've": "should have",
  "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
  "so's": "so is", "that'd": "that would", "that'd've": "that would have", "that's": "that is",
  "there'd": "there had", "there'd've": "there would have", "there's": "there is",
  "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
  "they'll've": "they will have", "they're": "they are", "they've": "they have",
  "to've": "to have", "wasn't": "was not", "we'd": "we had", "we'd've": "we would have",
  "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
  "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
  "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
  "when've": "when have", "where'd": "where did", "where's": "where is",
  "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is",
  "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
  "won't": "will not", "won't've": "will not have", "would've": "would have",
  "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
  "y'alls": "you alls", "y'all'd": "you all would", "y'all'd've": "you all would have",
  "y'all're": "you all are", "y'all've": "you all have", "you'd": "you had",
  "you'd've": "you would have", "you'll": "you you will", "you'll've": "you you will have",
  "you're": "you are", "you've": "you have"
}

def normalize_text(text):
    """
    文字列の正規化と短縮形の展開を行う。
    """
    if text is None: return ""
    text = str(text)

    # Retain only alphanumeric characters and specific punctuations
    text = re.sub(r'[^a-zA-Z0-9\s\'.,?!]', '', text)
    # Collapse multiple consecutive punctuations into one
    text = re.sub(r'([.,?!])\1+', r'\1', text)
    # Remove consecutive newlines and multiple spaces
    text = re.sub(r'\n{2,}', '\n', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize by word boundaries
    words = re.findall(r"\b\w+[']?\w*\b|[^\s\w]+", text)
    expanded_words = []

    for word in words:
        lower_word = word.lower()
        if lower_word in cList:
            # Expand contraction
            expanded = cList[lower_word]
            # Match original capitalization if necessary
            if word[0].isupper():
                expanded = expanded.capitalize()
            expanded_words.append(expanded)
        else:
            expanded_words.append(word)

    return " ".join(expanded_words)

## 4. Token Length Filtering & Label Split
BERTの入力制限（最大512トークン）に合わせ、超過するシーケンスを除外します。その後、ラベル（toxic）に応じてデータセットを分割します。

In [ ]:
def divide(file):
    """
    データセットを読み込み、正規化、トークン長によるフィルタリング、
    およびラベルベースでの分割を実行する。
    """
    # 1. Load dataset and apply normalization
    df_workbook = pd.read_csv(file + '.csv')
    df_workbook['cleaned_text'] = df_workbook['comment_text'].apply(normalize_text)

    # 2. Calculate token length using BERT tokenizer
    df_workbook['token_len'] = df_workbook['cleaned_text'].apply(lambda x: len(tokenizer.encode(x, add_special_tokens=True)))

    # 3. Filter out sequences exceeding maximum sequence length (512 tokens)
    df_workbook = df_workbook[df_workbook['token_len'] < 512].reset_index(drop=True)

    # 4. Split data based on 'toxic' label
    df_safe = df_workbook[df_workbook['toxic'] == 0].reset_index(drop=True)
    df_out = df_workbook[df_workbook['toxic'] == 1].reset_index(drop=True)
    
    # Export processed datasets
    if file == "train":
      df_safe.to_csv(file+"_safe.csv", index=False)
      df_out.to_csv(file+"_out.csv", index=False)
    elif file == "test_new":
      df_safe.to_csv("test_safe.csv", index=False)
      df_out.to_csv("test_out.csv", index=False)

## 5. Execution
定義したパイプラインを使用して、テストデータと学習データの前処理を実行します。

In [ ]:
# Process test dataset
divide("test_new")

# Process train dataset
divide("train")